In [1]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../src')

%load_ext autoreload
%autoreload 2

import torch
import torch_geometric
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt


from torch_geometric.datasets import TUDataset
from preprocessing import data_transformation
from similarity import calculate_similarity_matrix

from model import GCN

In [3]:
dataset = TUDataset(root='datasets/', name='MUTAG')
torch.manual_seed(1234)
dataset = dataset.shuffle()

## Preprocessing

#### Split: Train test validation

```train_dataset```: for training model<br/>
```val_dataset```: evaluate model for hyperparameter tunning<br/>
```test_dataset```: testing model after complete training<br/>

In [4]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

In [7]:
tr, ts, vl = 0.8, 0.1, 0.1
dslen = len(dataset)
tri = round(tr*dslen)
tsi = round((tr+ts)*dslen)
train_dataset = dataset[:tri]
test_dataset = dataset[tri:tsi]
val_dataset = dataset[tsi:]

In [10]:
dataset.y

tensor([1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1])

In [11]:
print(len(train_dataset))
train_dataset.y

150


tensor([1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 1])

In [12]:
len(test_dataset)
test_dataset.y

tensor([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0])

In [13]:
len(val_dataset)
val_dataset.y

tensor([0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1])

#### Batching

In [14]:
# paper 128
batch_size = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Building Model

In [20]:
from torch_geometric.nn import GCNConv
from torch.nn import Linear

from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_add_pool

#### Train

In [22]:
def train_base(model, loader, experiment_mode=False):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
    
    model.train()
    
    for data in loader:
        if experiment_mode:
            emb, h = model(data.x, data.edge_index, data.batch, data.ptr)
        else:
            emb, h = model(data.x, data.edge_index, data.batch)
        loss = criterion(h, data.y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return loss
    #     print(h[0])|
    # print(loss)

@torch.no_grad()
def test_base(model, loader, experiment_mode=False):
    model.eval()
    correct = 0
    for data in loader:
        if experiment_mode:
            emb, h = model(data.x, data.edge_index, data.batch, data.ptr)
        else:
            emb, h = model(data.x, data.edge_index, data.batch)
        pred = h.argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct/len(loader.dataset)

### Experiment Model

In [26]:
from sklearn.cluster import AffinityPropagation

#### Model modification

In [124]:
# paper 128
batch_size = 10

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

batch1 = next(iter(train_loader))

In [127]:


from similarity import calculate_similarity_matrix, testt


# AP Clustering
from sklearn.cluster import AffinityPropagation

from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_max_pool

import torch.nn.functional as F

class Experiment(torch.nn.Module):
    # merging type: o --> complement only, s --> substraction, c --> concatenation
    def __init__(self, dataset, hidden_channels, k = 1):
        super(Experiment, self).__init__()
        
        # save number of subgraphs, default 1
        self.k_subgraph = k
        
        # weight seed
        torch.manual_seed(42)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        # self.conv3 = GCNConv(hidden_channels, hidden_channels)
        
        # embeddings for subgraph
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.conv5 = GCNConv(hidden_channels, hidden_channels)
        
        # attention layer
        self.query_layer = Linear(hidden_channels,hidden_channels)
        self.key_layer = Linear(hidden_channels,hidden_channels)
        self.value_layer = Linear(hidden_channels,hidden_channels)
        
        # classification layer
        self.lin = Linear(hidden_channels*2, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch, ptr):
        # Embed original
        embedding = self.conv1(x, edge_index)
        embedding = embedding.relu()
        embedding = self.conv2(embedding, edge_index)
        # embedding = embedding.relu()
        # embedding = self.conv3(embedding, edge_index)
        # embedding = embedding.relu()
        
        # generate subgraph based on embeddings
        feature_emb = embedding.detach()
        
        subgraph_edge_index, communities, S, batch_communities = self.subgraph_generator(feature_emb, edge_index, batch, ptr)
        subgraph_embedding = self.conv4(embedding, subgraph_edge_index)
        subgraph_embedding = subgraph_embedding.relu()
        subgraph_embedding = self.conv5(subgraph_embedding, subgraph_edge_index)
        # subgraph_embedding = subgraph_embedding.relu()
        
        # apply readout layer/pooling for each subgraphs
        subgraph_pool_embedding = self.subgraph_pooling(subgraph_embedding, communities, batch, ptr, batch_communities)
        
        # apply selective (top k) attention
        topk_subgraph_embedding = self.selectk_subgraph(embedding, subgraph_pool_embedding, self.k_subgraph)
        
        # print('get topk subgraph embedding', topk_subgraph_embedding)
        embedding = global_mean_pool(embedding, batch)
        
        
        subgraph_embedding = global_max_pool(subgraph_embedding, batch)
        
        print('shape subgraph embedding ', subgraph_embedding.shape)
        print('shape topk_subgraph_embedding ', topk_subgraph_embedding.shape)
        print('shape topk_subgraph_embedding transformed', topk_subgraph_embedding.view(len(embedding), -1).shape)
        
        # print('pre transform (original subgraph_embedding)', subgraph_embedding)
        # print('pre transform topk subgraph embedding', topk_subgraph_embedding.view(2, -1))
        # h = torch.cat((embedding, topk_subgraph_embedding.view(2, -1)), 1)
        # h = torch.cat((embedding, subgraph_embedding), 1)
        combined_embeddings = torch.cat((embedding, topk_subgraph_embedding.view(len(embedding), -1)), 1)
        
        
        h = F.dropout(combined_embeddings, p=0.3, training=self.training)
        h = self.lin(h)
        h = h.relu()
        h = F.dropout(h, p=0.3, training=self.training)
        h = self.lin2(h)
        
        return embedding, h, S, communities, topk_subgraph_embedding.view(len(embedding), -1)
    
    def selectk_subgraph(self, embs, sub_embs, k = 1):
        # calculate attention and select top k subgraph
        
        topk_subgraphs_all = []

        for i, (emb, sub_emb) in enumerate(zip(embs, sub_embs)):
            sub = torch.tensor(sub_emb)
            sub = sub.to(torch.float32)

            # transform
            query = self.query_layer(emb)
            key = self.key_layer(sub)
            value = self.value_layer(sub)

            # att score
            attention_score = torch.matmul(query, key.transpose(0,1))
            attention_weight = F.softmax(attention_score, dim=0)

            print(f'attention score {attention_score}')
            print(f'attention weight {attention_weight}')

            print('select top k')
            topk_subgraph_embeddings = None
            
            if (k <= len(sub)):
                topk_scores, topk_indices = torch.topk(attention_weight, k)
                topk_subgraph_embeddings = sub[topk_indices]
            else:
                print('too big')
                
            topk_subgraphs_all.append(topk_subgraph_embeddings)
        
        return torch.stack(topk_subgraphs_all)
    
    def subgraph_generator(self, embeddings, batch_edge_index, batch, ptr):
        '''
        Return subgraph_edge_index (edge_index of created subgraph)
        '''
        graph_counter = 0
        edge_index = [[],[]]
        subgraph_edge_index = [[],[]]
        # Gs = []
        sub_created = False
        graph_bound = {}
        all_communities = []
        batch_communities = {}
        S = []

        for i in range(len(ptr)-1):
            graph_bound[i] = [ptr[i].item(), ptr[i+1].item()]
        
        for i, (src, dst) in enumerate(zip(batch_edge_index[0], batch_edge_index[1])):
            lower_bound = graph_bound[graph_counter][0]
            upper_bound = graph_bound[graph_counter][1]
            if ((src >= lower_bound and src < upper_bound) or
                (dst >= lower_bound and dst < upper_bound)):
                
                edge_index[0].append(src - lower_bound)
                edge_index[1].append(dst - lower_bound)
            else:
                sub_created = True
                
            if (i == len(batch_edge_index[0]) - 1) or sub_created:
                sub_created = False
                
                embs = []
                # make new graph
                for i, (b, emb) in enumerate(zip(batch, embeddings)):
                    if (b == graph_counter):
                        embs.append(emb)
                
                G = data_transformation(edge_index, embs)
                # dont need this at the moment
                # Gs.append(G)
                
                # Calculate similarity matrix
                S = calculate_similarity_matrix(G)
                
                # AP Clustering        
                clustering = AffinityPropagation(affinity='precomputed', damping=0.8, random_state=42, convergence_iter=15, max_iter=1000)
                clustering.fit(S)
                
                print('clustering label', clustering.labels_)
                
                # Get community
                communities = {}
                for lab in clustering.labels_:
                    communities[lab] = []
                    all_communities.append(lab)
                for nd, clust in enumerate(clustering.labels_):
                    communities[clust].append(nd)
                print(communities)
                
                edge_index = [[],[]]
                batch_communities[graph_counter] = communities
                
                graph_counter+=1
                
                # Make subgraph edge_index
                for c in communities:
                    w = G.subgraph(communities[c])
                    for sub in w.edges:
                        subgraph_edge_index[0].append(sub[0] + lower_bound)
                        subgraph_edge_index[1].append(sub[1] + lower_bound)
                
                print()
                # break # sementara aja
        
        # print('batch communities', batch_communities)
        return torch.tensor(subgraph_edge_index), all_communities, S, batch_communities
    
        
    def subgraph_pooling(self, embeddings, communities, batch, ptr, batch_communities):
        # batch communities: batch (or graph in this batch) -> communities -> member
        pool_type = 'mean'
        curr_batch = 0
        emb_temp = None
        emb_pool = []
        all_emb_pool = []
        print('batch communities', batch_communities)
        
        print('batch loop')
        print('')
        
        # LOOP THROUGH BATCH
        for b in batch_communities:
            print(f'==== BATCH {b} ====')
            print('lower bound', ptr[b].item())
            print('len communities on this batch', len(batch_communities[b]))
            
            # initialize array
            emb_temp = [[] for _ in range(len(batch_communities[b]))]
            emb_pool = [[] for _ in range(len(batch_communities[b]))]
            for comm in batch_communities[b]:
                for member in batch_communities[b][comm]:
                    # emb_temp[comm].append(member + ptr[b].item())
                    index_used = member + ptr[b].item()
                    emb_temp[comm].append(embeddings[index_used].detach().tolist())
                    # print('embtemp-log', emb_temp)
                    # print(comm, "-",member)
                print('break new community', comm)

                # Pooling per sub graph                
                if pool_type == 'mean': # mean pool
                    emb_pool[comm] = np.array(emb_temp[comm]).mean(axis=0)
                elif pool_type == 'add': # add pool
                    emb_pool[comm] = np.array(emb_temp[comm]).sum(axis=0)
                else:
                    print('TODO: fill later')
                
            print("pool subgraph === ", np.array(emb_pool))
            print("Pool size ", np.array(emb_pool).shape)
            print()
            all_emb_pool.append(emb_pool.copy())
        
        print('')
        print('====== ALL SUBGRAPH POOLING RESULT ======')
        print(all_emb_pool)
        
        return all_emb_pool

experiment = Experiment(dataset, 64)
emb, h, S, communities, sub_emb = experiment(batch1.x, batch1.edge_index, batch1.batch, batch1.ptr)

clustering label [0 0 0 0 0 0 1 0 0 1 2 2 2 2 2 1 2]
{0: [0, 1, 2, 3, 4, 5, 7, 8], 1: [6, 9, 15], 2: [10, 11, 12, 13, 14, 16]}

clustering label [0 1 1 3 0 0 2 2 2 2 2 3 3 1 3 3 3 3 3 2 1 3 3]
{0: [0, 4, 5], 1: [1, 2, 13, 20], 3: [3, 11, 12, 14, 15, 16, 17, 18, 21, 22], 2: [6, 7, 8, 9, 10, 19]}

clustering label [0 0 0 0 1 1 1 2 2 2 2 2 2 2 2 2 2]
{0: [0, 1, 2, 3], 1: [4, 5, 6], 2: [7, 8, 9, 10, 11, 12, 13, 14, 15, 16]}

clustering label [0 0 0 0 1 1 1 1 1 2 1 1 0 0 3 1 2 2 3 3 2 1 1]
{0: [0, 1, 2, 3, 12, 13], 1: [4, 5, 6, 7, 8, 10, 11, 15, 21, 22], 2: [9, 16, 17, 20], 3: [14, 18, 19]}

clustering label [0 1 1 1 0 0 0 3 1 1 2 2 2 3 3 2]
{0: [0, 4, 5, 6], 1: [1, 2, 3, 8, 9], 3: [7, 13, 14], 2: [10, 11, 12, 15]}

clustering label [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1]
{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 1: [10, 11, 12, 13, 14, 15, 16]}

clustering label [0 0 0 0 2 2 1 1 1 1 2 0 0 2 1 1]
{0: [0, 1, 2, 3, 11, 12], 2: [4, 5, 10, 13], 1: [6, 7, 8, 9, 14, 15]}

clustering label [1 0 0 0 2 2 2 0 

In [138]:
h

tensor([[ 0.1253, -0.0275],
        [ 0.1351, -0.0011],
        [ 0.1076, -0.0756],
        [ 0.1343, -0.0447],
        [ 0.1359, -0.0559],
        [ 0.1900, -0.0916],
        [ 0.1417, -0.0782],
        [ 0.1174, -0.0663],
        [ 0.1419, -0.0372],
        [ 0.1469, -0.0010]], grad_fn=<AddmmBackward0>)

In [66]:
def selectk_subgraph():
    query_layer = Linear(64,64)
    key_layer = Linear(64,64)
    value_layer = Linear(64,64)
    sub = torch.tensor(sub_emb[0])
    sub = sub.to(torch.float32)

    # transform
    query = query_layer(emb[0])
    key = key_layer(sub)
    value = value_layer(sub)

    # att score
    attention_score = torch.matmul(query, key.transpose(0,1))
    attention_weight = F.softmax(attention_score, dim=0)

    print(f'attention score {attention_score}')
    print(f'attention weight {attention_weight}')

    print('select top k')
    topk_subgraph_embeddings = None
    k = 2
    if (k < len(sub)):
        topk_scores, topk_indices = torch.topk(attention_weight, k)
        topk_subgraph_embeddings = sub[topk_indices]
    else:
        print('too big')
    return topk_subgraph_embeddings
    print(topk_subgraph_embeddings)
tk = selectk_subgraph()

attention score tensor([0.0384, 0.0401, 0.0506], grad_fn=<SqueezeBackward4>)
attention weight tensor([0.3318, 0.3324, 0.3359], grad_fn=<SoftmaxBackward0>)
select top k


In [99]:
tks = torch.stack([tk, tk])
print(tks)
print(tks.view(2, -1))

tensor([[[-0.0899, -0.1282,  0.0192,  0.0541, -0.0026,  0.0289, -0.1105,
           0.1363, -0.0516,  0.0120,  0.0660, -0.0674,  0.0270,  0.1252,
          -0.0760,  0.0043,  0.1235,  0.0869,  0.0051, -0.1587, -0.0569,
           0.0846,  0.0735,  0.1890, -0.0159,  0.1254,  0.0133, -0.0498,
           0.1450, -0.2546, -0.0205, -0.0333,  0.0164,  0.0015,  0.0234,
           0.0311, -0.1800,  0.0291, -0.1397,  0.0814,  0.0354,  0.1323,
           0.0760,  0.1478, -0.0086,  0.0600,  0.0144,  0.1243, -0.0531,
           0.2455, -0.1247, -0.0571,  0.0749,  0.1452,  0.0883,  0.0319,
           0.0807, -0.0748, -0.0405, -0.1101,  0.0264, -0.0398, -0.0205,
           0.2983],
         [-0.0753, -0.1144,  0.0271,  0.0615, -0.0065,  0.0311, -0.1082,
           0.1212, -0.0399,  0.0299,  0.0830, -0.0601,  0.0067,  0.1248,
          -0.0647,  0.0151,  0.1325,  0.0685,  0.0026, -0.1387, -0.0442,
           0.0874,  0.0781,  0.1792, -0.0068,  0.1261,  0.0045, -0.0527,
           0.1178, -0.2363, -0.

In [229]:
print(len(communities) == 
batch1.batch.size()[0])

True


In [207]:
clustering = AffinityPropagation(affinity='precomputed', damping=0.7, random_state=42, convergence_iter=15, max_iter=3000)
clustering.fit(S)
clustering.labels_
# clustering.

array([0, 0, 0, 0, 3, 0, 1, 1, 2, 1, 2, 2, 3, 3, 3, 4, 3, 3, 3, 3, 4, 4,
       4], dtype=int64)

In [208]:
dataset

MUTAG(188)

In [210]:
def expTrain(train_loader, val_loader, test_loader, epoch = 2):
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning) 

    experiment = Experiment(dataset, 64)

    # Train
    print('process training')
    for _ in range(epoch):
        loss = round(train_base(experiment, train_loader, True).item(), 5)
        train_acc = round(test_base(experiment, train_loader, True), 5)
        val_acc = round(test_base(experiment, val_loader, True), 5)
        
        print(f'epoch {_}; loss: {loss}; train_acc: {train_acc}; test_acc: {val_acc}')

    # Test
    print('process testing')
    test = test_base(experiment, test_loader, True)
    print(f'Accuracy: {test}')

# expTrain(train_loader, val_loader, test_loader, epoch = 1)

In [211]:
def baseTrain(train_loader, val_loader, test_loader, epoch = 10):
    base = Base(dataset, 64)

    # Train
    for _ in range(epoch):
        loss = round(train_base(base, train_loader).item(), 5)
        train_acc = round(test_base(base, train_loader), 5)
        val_acc = round(test_base(base, val_loader), 5)
        
        print(f'epoch {_}; loss: {loss}; train_acc: {train_acc}; val_acc: {val_acc}; test: {round(test_base(base, test_loader), 2)}')

    # Test
    test = test_base(base, test_loader)
    print(f'Accuracy: {test}')

#### Cross validation 10

In [30]:
from sklearn.model_selection import KFold

In [45]:
train_dataset = dataset[:round(len(dataset) * 0.8)]
test_dataset = dataset[round(len(dataset) * 0.8):]

In [ ]:
S

In [46]:
# 
train_dataset
test_dataset
k = 10

splits = KFold(n_splits=k,shuffle=True,random_state=42)
k_counter = 0

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(train_dataset)))):
    # print('Fold {}'.format(fold + 1))
    # print(f'Fold',fold,'Train_idx',train_idx,'Val_idx',val_idx)
    print(f'Fold {fold}/{k}')
    #if k_counter > 2:
    #    break
    
    fold_train = []
    for key in train_idx:
        fold_train.append(train_dataset[key])

    fold_val = [] 
    for key in val_idx:
        fold_val.append(train_dataset[key])

    tr = DataLoader(fold_train, batch_size=batch_size, shuffle=True)
    vd = DataLoader(fold_val, batch_size=batch_size, shuffle=True)
    ts = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
    # Base model
    print("=== Base model ===")
    baseTrain(tr, vd, ts, 10)
    print("=== Experiment model ===")
    expTrain(tr, vd, ts, 10)
    
    k_counter += 1

Fold 0/10
=== Base model ===
epoch 0; loss: 0.69358; train_acc: 0.7125; val_acc: 0.675; test: 0.81
epoch 1; loss: 0.51878; train_acc: 0.72083; val_acc: 0.675; test: 0.73
epoch 2; loss: 0.57897; train_acc: 0.71944; val_acc: 0.6375; test: 0.73
epoch 3; loss: 0.55901; train_acc: 0.72778; val_acc: 0.6875; test: 0.74
epoch 4; loss: 0.58857; train_acc: 0.70556; val_acc: 0.575; test: 0.68
epoch 5; loss: 0.49834; train_acc: 0.7125; val_acc: 0.6625; test: 0.72
epoch 6; loss: 0.55777; train_acc: 0.73472; val_acc: 0.7125; test: 0.77
epoch 7; loss: 0.59016; train_acc: 0.74306; val_acc: 0.725; test: 0.76
epoch 8; loss: 0.52072; train_acc: 0.74722; val_acc: 0.6625; test: 0.77
epoch 9; loss: 0.58987; train_acc: 0.7125; val_acc: 0.75; test: 0.77
Accuracy: 0.77
=== Experiment model ===
process training
epoch 0; loss: 0.67796; train_acc: 0.475; test_acc: 0.575
epoch 1; loss: 0.66832; train_acc: 0.65417; test_acc: 0.7125
epoch 2; loss: 0.7253; train_acc: 0.60972; test_acc: 0.6625
epoch 3; loss: 0.65472; 